# Options for Loading Multiple BOTPT 15s Files

This notebook demos two different options for loading all of the BOTPT 15-second Nano files located in the kdata directory (covers approximately 7 years worth of data):

* using list comprehension and a small processing routine to load and lightly process each file individually before concatenating the entire collection into a single xarray file
* using the ooi_data_explorations load_kdata utility to load the files, where the process is essentially identical to the list comprehension but it handles things a little differently behind the scenes

A third option, using the `open_mfdataset` was also explored, but it did not work. There are two key issues with using `open_mfdataset` with OOI datasets. First, we use `obs` as the dimension variable, rather than `time`. For each NetCDF file, the `obs` index starts at 0 and increments up from there. The `open_mfdataset` will merge all the files based on that dimension if you don't tell it to do otherwise. All the 0's get merged, all the 1's get merged, etc., and the user ends up with a `what?` dataset. The second issue is time. The time record needs to be monotonically increasing in order for the concatentation and merge routines used by `open_mfdataset` to work. For uncabled systems, we often have overlapping deployments (new mooring gets deployed before the old one is recovered for approximately 24 to 72 hours) resulting in a non-monotonically increasing time record during the overlap periods. For cabled datasets the issue is a little different. Often times the last record in one NetCDF file is duplicated as the first record in the next NetCDF file (a known bug). Again, the `time` dimension is no longer monotonically increasing and `open_mfdataset` fails.

In [1]:
# import modules needed in subsequent processing
import os
import glob
import numpy as np
import xarray as xr

from ooi_data_explorations.common import load_kdata

NetrcParseError: bad toplevel token 'touch' (/home/jovyan/.netrc, line 1)

In [2]:
# define the dataset we are looking to access, will use the elements listed below to either create a list of
# files of interest, or as inputs to the load_kdata function
site = 'RS03ASHS'
node = 'MJ03B'
sensor = '09-BOTPTA304'
method = 'streamed'
stream = 'botpt_nano_sample_15s'
tag = '*botpt_nano_sample_15s*.nc'  # note, this tag is critical (see below), ensures we only load the 15-second files

In [4]:
# create the dataset ID based on the site, node, sensor, method and stream names listed above
dataset_id = '-'.join([site, node, sensor, method, stream])

# construct the kdata directory path from the dataset ID
kdata = os.path.abspath(os.path.join(os.path.expanduser('~'), 'ooi/kdata'))
kdata = os.path.abspath(os.path.join(kdata, dataset_id))

# Create a list of the files using globbing
files = glob.glob(kdata + '/' + tag)
files

# note, the tag used here is critical. The 15-second files are created from the full resolution files. Any time a dataset
# is derived from another one, or uses a variable from another one (e.g., salinity from a co-located CTD), those source
# datasets will be included in the kdata directory. If we just used a tag of '*.nc', we would get all of the NetCDF files;
# the 15-second resolution files AND the full resolution files. If we tried to load all of those full resolution files,
# this whole example would fail and the kernel would crash!

['/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s/deployment0001_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20230410T082715-20230411T082700.nc',
 '/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s/deployment0001_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20230211T105900-20230212T110530.nc',
 '/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s/deployment0001_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20221026T074400-20221027T085645.nc',
 '/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s/deployment0001_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20240117T090500-20240118T091345.nc',
 '/home/jovyan/ooi/kdata/RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s/deployment0001_RS03ASHS-MJ03B-09-BOTPTA304-streamed-botpt_nano_sample_15s_20221012T073715-20221013T080730.nc',
 '/home/jovyan/ooi/kdata/

## Option 1: List Comprehesion

In [5]:
# create a function to handle each file, loading, resetting dimensions, cleaning up some variables, etc
def proc_botpt(file):
    # load the file and reset the dimensions
    data = xr.open_dataset(file, chunks=1000)
    data = data.swap_dims({'obs': 'time'})
    data = data.reset_coords()
    
    # drop select variables
    keys = ['obs', 'id', 'provenance', 'driver_timestamp', 'ingestion_timestamp', 'port_timestamp', 'preferred_timestamp']
    for key in keys:
        if key in data.variables:
            data = data.drop_vars(key)

    # return the dataset for further processing
    return data

In [12]:
%%time
# load the data from each file into a list of datasets
data = [proc_botpt(file) for file in files]

CPU times: user 57.2 s, sys: 7.68 s, total: 1min 4s
Wall time: 1min 12s


In [13]:
data[0]

<xarray.Dataset>
Dimensions:                        (time: 5760)
Coordinates:
  * time                           (time) datetime64[ns] 2023-04-10T08:27:15 ...
Data variables: (12/18)
    botsflu_5minrate               (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_time15s                (time) datetime64[ns] dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meandepth_qc_results   (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_predtide               (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meanpres               (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_5minrate_qc_executed   (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    ...                             ...
    botsflu_predtide_qc_executed   (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meandepth_qc_executed  (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    deployment                     (time) int32 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_5minrate_qc_results    (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_10minrate              (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meandepth              (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes: (12/70)
    node:                               MJ03B
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  streamed
    stream:                             botpt_nano_sample_15s
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                45.933638
    lon:                                -130.013897

In [14]:
%%time
# concatenate the list of datasets into a single dataset
data = xr.concat(data, dim='time', data_vars='minimal', coords='minimal')
data = data.sortby('time')
_, index = np.unique(data['time'], return_index=True)
data = data.isel(time=index)

# note, sometimes the last record in one NetCDF file is the same as the first record in the next NetCDF file. This is a known bug in the system. We use the np.unique function to make sure we are removing those duplicate records

CPU times: user 52.4 s, sys: 9.99 s, total: 1min 2s
Wall time: 1min 2s


In [15]:
data

<xarray.Dataset>
Dimensions:                        (time: 12874240)
Coordinates:
  * time                           (time) datetime64[ns] 2017-08-15T00:10:30 ...
Data variables: (12/18)
    botsflu_5minrate               (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_time15s                (time) datetime64[ns] dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meandepth_qc_results   (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_predtide               (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meanpres               (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_5minrate_qc_executed   (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    ...                             ...
    botsflu_predtide_qc_executed   (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meandepth_qc_executed  (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    deployment                     (time) int32 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_5minrate_qc_results    (time) uint8 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_10minrate              (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
    botsflu_meandepth              (time) float64 dask.array<chunksize=(1000,), meta=np.ndarray>
Attributes: (12/70)
    node:                               MJ03B
    comment:                            
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  streamed
    stream:                             botpt_nano_sample_15s
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                45.933638
    lon:                                -130.013897

## Option 2: Using load_kdata

In [16]:
%%time
data = load_kdata(site, node, sensor, method, stream, tag, use_dask=True)  # note, chunk sizes set to 'auto' in this case

Loading and Processing Data Files: 100%|██████████| 473/473 [01:17<00:00,  6.12it/s]
CPU times: user 1min 59s, sys: 43.4 s, total: 2min 43s
Wall time: 2min 13s


In [17]:
data

<xarray.Dataset>
Dimensions:                        (time: 12874240)
Coordinates:
  * time                           (time) datetime64[ns] 2017-08-15T00:10:30 ...
Data variables: (12/18)
    botsflu_5minrate               (time) float64 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_time15s                (time) datetime64[ns] dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_meandepth_qc_results   (time) uint8 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_predtide               (time) float64 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_meanpres               (time) float64 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_5minrate_qc_executed   (time) uint8 dask.array<chunksize=(10175088,), meta=np.ndarray>
    ...                             ...
    botsflu_predtide_qc_executed   (time) uint8 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_meandepth_qc_executed  (time) uint8 dask.array<chunksize=(10175088,), meta=np.ndarray>
    deployment                     (time) int32 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_5minrate_qc_results    (time) uint8 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_10minrate              (time) float64 dask.array<chunksize=(10175088,), meta=np.ndarray>
    botsflu_meandepth              (time) float64 dask.array<chunksize=(10175088,), meta=np.ndarray>
Attributes: (12/69)
    node:                               MJ03B
    comment:                            Data produced by the OOI M2M API and ...
    publisher_email:                    
    sourceUrl:                          http://oceanobservatories.org/
    collection_method:                  streamed
    stream:                             botpt_nano_sample_15s
    ...                                 ...
    geospatial_lon_resolution:          0.1
    geospatial_vertical_units:          meters
    geospatial_vertical_resolution:     0.1
    geospatial_vertical_positive:       down
    lat:                                45.933638
    lon:                                -130.013897